In [1]:
import json, requests
from pprint import *
import csv
import collections
from unidecode import unidecode
import re, requests
from rdflib import Dataset, URIRef, Literal, Namespace, RDF, RDFS, OWL, XSD
from iribaker import to_iri
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
from geomet import wkt
# pip install git+git://github.com/geomet/geomet.git

In [3]:
url = 'http://geodata.nationaalgeoregister.nl/nwbwegen/wfs?service=WFS&request=GetFeature&typeName=nwbwegen:wegvakken&count=%d&startIndex=%d&outputFormat=json&srsName=EPSG:4326'

count = 10
startIndex = 0
amsterdamData = []

In [4]:
def retreiveWegvakken():
    count = 10000
    startIndex = 1
    f = open('amsterdam-wegvakken.csv','wb')
    resp = requests.get(url=url % (count, startIndex))
    data = json.loads(resp.text)

    featuresChecked = 0
    totalFeatures = data['totalFeatures']
    print totalFeatures

    keys = ['coordinates', 'avgCoordinates', 'wtk']
    keys += data['features'][0]['properties'].keys()
    print keys
    w = csv.DictWriter(f,keys)
    w.writeheader()

    startIndex = 1
    filterCity = 'Amsterdam'
    while startIndex < totalFeatures:
        retry = True
        while retry:
            try:
                resp = requests.get(url=url % (count, startIndex))
                retry = False
            except Timeout:
                print "retry, connection timed out"
            except Exception:
                print "some other error, Exit!"
                return
        data = json.loads(resp.text)
        print startIndex
        for feat in data['features']:
            if feat['properties']['gme_naam'] == filterCity:
                wegVakData = {}
                coordinates = feat['geometry']['coordinates']
                avgCoordinates = [sum(i[0] for i in coordinates[0])/len(coordinates[0]), sum(i[1] for i in coordinates[0])/len(coordinates[0])]
                wegVakData = feat['properties'].copy()
                wegVakData['wtk'] = feat['geometry']
                wegVakData['coordinates']    = coordinates
                wegVakData['avgCoordinates'] = avgCoordinates
                wegVakData =  dict([(k.encode('ascii', 'replace'), v.encode('ascii', 'replace') if type(v) is str or type(v) is unicode else v) for k, v in wegVakData.items()])
                w.writerow(wegVakData)
        startIndex += count

    f.close()

In [20]:
TUTORIAL_REPOSITORY = "http://stardog.krw.d2s.labs.vu.nl/group12"

def upload_to_stardog(data):
    transaction_begin_url = TUTORIAL_REPOSITORY + "/transaction/begin"
    
    # Start the transaction, and get a transaction_id
    response = requests.post(transaction_begin_url, headers={'Accept': 'text/plain'})
    transaction_id = response.content
    print response.status_code
    print response.headers
    
    
    # POST the data to the transaction
    post_url = TUTORIAL_REPOSITORY + "/" + transaction_id + "/add"
    response = requests.post(post_url, data=data, headers={'Accept': 'text/plain', 'Content-type': 'application/trig'})
    print response.status_code
    print response.headers
    
    
    # Close the transaction
    transaction_close_url = TUTORIAL_REPOSITORY + "/transaction/commit/" + transaction_id
    response = requests.post(transaction_close_url)
    print response.status_code
    print response.headers
    
    return str(response.status_code)

In [6]:
VALID_CHARS = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-._~:/?#[]@!$&'()*+,;=")

In [7]:
wegvakken = csv.DictReader(open('amsterdam-wegvakken.csv', 'r'))

In [8]:
# A namespace for our resources
data = 'http://data.krw.d2s.labs.vu.nl/group12/resource/'
DATA = Namespace(data)
# A namespace for our vocabulary items (schema information, RDFS, OWL classes and properties etc.)
vocab = 'http://data.krw.d2s.labs.vu.nl/group12/vocab/'
VOCAB = Namespace('http://data.krw.d2s.labs.vu.nl/group12/vocab/')

# The URI for our graph
graph_uri = URIRef('http://data.krw.d2s.labs.vu.nl/group12/resource/roadsectiongraph')

# We initialize a dataset, and bind our namespaces
dataset = Dataset()
dataset.bind('g12data', DATA)
dataset.bind('g12vocab', VOCAB)

# We then get a new graph object with our URI from the dataset.
graph = dataset.graph(graph_uri)

# Create namespaces for our Geo-data
GSP = Namespace('http://www.opengis.net/ont/geosparql#')
GSF = Namespace('http://www.opengis.net/ont/sf#')
prov = Namespace('http://www.w3.org/ns/prov#')


In [23]:
# Load the externally defined schema into the default graph (context) of the dataset
dataset = Dataset()
dataset.bind('g12data',DATA)
dataset.bind('g12vocab',VOCAB)
dataset.default_context.parse('vocab.ttl', format='turtle')
graph = dataset.graph(graph_uri)
# create pattern for splitting streets in Kruispunt column
splitpattern = re.compile(r' / *| - ')

uploadCount = 0
for row in wegvakken:
    # first create the URI's
    wegvak = URIRef(to_iri(data + 'RoadSection/' + row['wvk_id']))
    jte_beg = URIRef(to_iri(data + 'Junction/' + row['jte_id_beg']))
    jte_end = URIRef(to_iri(data + 'Junction/' + row['jte_id_end']))
    hnrstrrhts = Literal(row['hnrstrrhts'], datatype=XSD['string'])
    hnrstrlnks = Literal(row['hnrstrlnks'], datatype=XSD['string'])
    e_hnr_rhts = Literal(row['e_hnr_rhts'], datatype=XSD['int'])
    e_hnr_lnks = Literal(row['e_hnr_lnks'], datatype=XSD['int'])
    l_hnr_rhts = Literal(row['l_hnr_rhts'], datatype=XSD['int'])
    stt_naam = ''.join([c for c in row['stt_naam'].strip().replace(' ', '_') if c in VALID_CHARS])
    straatUri = URIRef(to_iri(data + stt_naam))
    gemeenteNaam = ''.join([c for c in row['gme_naam'].replace(' ', '_') if c in VALID_CHARS])
    dorp = URIRef(to_iri(data + gemeenteNaam))
    wegvak_MLS = URIRef(to_iri(data + 'RoadSection/' + row['wvk_id'] + '/MultiLineString'))
    jte_beg_Point = URIRef(to_iri(data + 'Junction/' + row['jte_id_beg'] + '/Point'))
    jte_end_Point = URIRef(to_iri(data + 'Junction/' + row['jte_id_end'] + '/Point'))

    #then the literals
    wegvakNaam = Literal('RoadSection' + row['wvk_id'], datatype=XSD['string'])
    wegvakid = Literal(row['wvk_id'], datatype=XSD['int'])
    straatNaam = Literal(row['stt_naam'].strip(), lang='nl')
    l_hnr_lnks = Literal(row['l_hnr_lnks'], datatype=XSD['int'])
    rijrichtng = Literal(row['rijrichtng'], datatype=XSD['string'])
    baansubsrt = Literal(row['baansubsrt'], datatype=XSD['string'])
    wegbehsrt = Literal(row['wegbehsrt'], datatype=XSD['string'])
    gemid = Literal(row['gme_id'], datatype=XSD['int'])
    gemeente = Literal(row['gme_naam'], lang='nl')
    jte_beg_id = Literal(row['jte_id_beg'], datatype=XSD['int'])
    jte_end_id = Literal(row['jte_id_end'], datatype=XSD['int'])
    jte_beg_naam = Literal('Junction' + row['jte_id_beg'], datatype=XSD['string'])
    jte_end_naam = Literal('Junction' + row['jte_id_end'], datatype=XSD['string'])
    #     avgCoordinates = json.loads(row['avgCoordinates'])
    #     lat = Literal(avgCoordinates[1], datatype=XSD['float'])
    #     lng = Literal(avgCoordinates[0], datatype=XSD['float'])
    #     coordinates = json.loads(row['coordinates'])[0]
    #     begLat = Literal(coordinates[0][1], datatype=XSD['float'])
    #     begLng = Literal(coordinates[0][0], datatype=XSD['float'])
    #     endLat = Literal(coordinates[-1][1], datatype=XSD['float'])
    #     endLng = Literal(coordinates[-1][0], datatype=XSD['float'])

    parsedCoordinates = wkt.dumps({'type': 'MultiLineString', 'coordinates': json.loads(row['coordinates'])})
    print parsedCoordinates
    coords = Literal(parsedCoordinates, datatype=GSP.wktLiteral)

    graph.add((wegvak, RDFS.label, wegvakNaam))
    graph.add((wegvak, RDF.type, VOCAB['RoadSection']))
    graph.add((wegvak, VOCAB['wvk_id'], wegvakid))
    #     graph.add((wegvak, VOCAB['lat'], lat))
    #     graph.add((wegvak, VOCAB['lng'], lng))
    graph.add((straatUri, RDF.type, VOCAB['Street']))
    graph.add((straatUri, RDFS.label, straatNaam))
    graph.add((wegvak, VOCAB['street'], straatUri))
    graph.add((dorp, RDF.type, VOCAB['Town']))
    graph.add((dorp, VOCAB['gme_naam'], gemeente))
    graph.add((dorp, VOCAB['gme_id'], gemid))
    graph.add((wegvak, VOCAB['town'], dorp))
    graph.add((wegvak, VOCAB['hnrstrrhts'], hnrstrrhts))
    graph.add((wegvak, VOCAB['hnrstrlnks'], hnrstrlnks))
    graph.add((wegvak, VOCAB['e_hnr_rhts'], e_hnr_rhts))
    graph.add((wegvak, VOCAB['e_hnr_lnks'], e_hnr_lnks))
    graph.add((wegvak, VOCAB['l_hnr_rhts'], l_hnr_rhts))
    graph.add((wegvak, VOCAB['l_hnr_lnks'], l_hnr_lnks))
    graph.add((wegvak, VOCAB['rijrichtng'], rijrichtng))
    graph.add((wegvak, VOCAB['wegbehsrt'], wegbehsrt))
    graph.add((jte_beg, RDF.type, VOCAB['Junction']))
    graph.add((jte_beg, RDFS.label, jte_beg_naam))
    graph.add((jte_beg, VOCAB['jte_id'], jte_beg_id))
    graph.add((jte_beg, GSP.hasGeometry, jte_beg_Point))
    #     graph.add((jte_beg, VOCAB['lat'], begLat))
    #     graph.add((jte_beg, VOCAB['lng'], begLng))
    graph.add((jte_end, RDF.type, VOCAB['Junction']))
    graph.add((jte_end, RDFS.label, jte_end_naam))
    graph.add((jte_end, VOCAB['jte_id'], jte_end_id))
    graph.add((jte_end, GSP.hasGeometry, jte_end_Point))
    #     graph.add((jte_end, VOCAB['lat'], endLat))
    #     graph.add((jte_end, VOCAB['lng'], endLng))
    graph.add((wegvak, VOCAB['junction_beg'], jte_beg))
    graph.add((wegvak, VOCAB['junction_end'], jte_end))
    graph.add((wegvak, GSP.hasGeometry, wegvak_MLS))
    graph.add((wegvak_MLS, RDF.type, GSF.MultiLineString))
    graph.add((wegvak_MLS, GSP.asWKT, coords))  # For some reason this triple causes problems for the triple store
    # print "skip", (wegvak_MLS, GSP.asWKT, coords)
    graph.add((jte_beg_Point, RDF.type, GSF.Point))
    graph.add((jte_end_Point, RDF.type, GSF.Point))
    print dataset.serialize(format='trig')
    print 'response.status_code = ' + upload_to_stardog(dataset.serialize(format='trig'))
    break
    count = count + 1
    if count % 400 == 0:
        #         print 'response.status_code = ' + upload_to_stardog(dataset.serialize(format='trig'))

        # We reset the dataset, and bind our namespaces
        dataset = Dataset()
        dataset.bind('g12data', DATA)
        dataset.bind('g12vocab', VOCAB)

        # We then get a new graph object with our URI from the dataset.
        graph = dataset.graph(graph_uri)


        # upload the final bit of data
        # print str(uploadCount) + ': response.status_code = ' + upload_to_stardog(dataset.serialize(format='trig'))

MULTILINESTRING ((4.8704575407052886 52.3709131772428833, 4.8717299082828962 52.3712244679152050))
@prefix : <http://data.krw.d2s.labs.vu.nl/group12/vocab#> .
@prefix g12data: <http://data.krw.d2s.labs.vu.nl/group12/resource/> .
@prefix g12vocab: <http://data.krw.d2s.labs.vu.nl/group12/vocab/> .
@prefix ns1: <http://www.opengis.net/ont/geosparql#> .
@prefix ns2: <urn:x-rdflib:> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

g12data:roadsectiongraph {
    <http://data.krw.d2s.labs.vu.nl/group12/resource/RoadSection/239374087> a g12vocab:RoadSection ;
        rdfs:label "RoadSection239374087"^^xsd:string ;
        g12vocab:e_hnr_lnks "0"^^xsd:int ;
        g12vocab:e_hnr_rhts "51"^^xsd:int ;
        g12vocab:hnrstrlnks "N"^^xsd:string ;
        g12vocab:hnrstrrhts "

200
{'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Keep-Alive': 'timeout=5, max=100', 'Server': 'nginx/1.4.6 (Ubuntu)', 'Connection': 'Keep-Alive', 'Date': 'Thu, 21 Apr 2016 09:12:01 GMT', 'Access-Control-Allow-Origin': '*', 'Content-Type': 'text/plain'}
500
{'Date': 'Thu, 21 Apr 2016 09:12:01 GMT', 'Content-Length': '0', 'Connection': 'close', 'Server': 'nginx/1.4.6 (Ubuntu)'}
response.status_code = 500


In [10]:
print coords

MULTILINESTRING ((4.7928810872111827 52.3533169280984296, 4.7934664900267299 52.3534456954132636, 4.7950266785200650 52.3537426492129256))


In [11]:
# print dataset.serialize(format='trig')

In [12]:
# with open('temp-roadsections-rdf.trig','w') as f:
#     dataset.serialize(f, format='trig')